## Test smells classification

**Task:** Classify a Java class as having one of the following smells: Conditional Test Logic or Exception Handling<br>
**Data:** Experiment data from https://testsmells.github.io/pages/research/experimentdata.html<br>
**Model:** LSTM neural network

### Load vectorized data

In [3]:
import pandas as pd

# Read input data
df1 = pd.read_csv('data/ConditionalTestLogic_vectors.csv')
df2 = pd.read_csv('data/ExceptionCatchingThrowing_vectors.csv')

# Get rid off useless features
df1.drop(df1.columns[[0, 1, 2]], axis=1, inplace=True)
df2.drop(df2.columns[[0, 1, 2]], axis=1, inplace=True)

# Preprocess
df1['Vector'] = [x[:-1] for x in df1['Vector']]
df2['Vector'] = [x[:-1] for x in df2['Vector']]
df1['Vector'] = [list(map(int, x.split(' '))) for x in df1['Vector']]
df2['Vector'] = [list(map(int, x.split(' '))) for x in df2['Vector']]
print(df1['Vector'])
print(df2['Vector'])

0       [618, 432, 2000, 46, 2001, 46, 2002, 46, 2003,...
1       [618, 432, 2000, 46, 2001, 46, 2002, 46, 2003,...
2       [432, 2000, 46, 2001, 46, 2002, 46, 2003, 46, ...
3       [432, 2000, 46, 2001, 46, 2002, 46, 2003, 46, ...
4       [432, 2000, 46, 2001, 46, 2002, 46, 2003, 46, ...
                              ...                        
1995    [618, 432, 2000, 46, 2001, 46, 2002, 46, 2003,...
1996    [432, 2000, 46, 2001, 46, 2002, 46, 2003, 46, ...
1997    [432, 2000, 46, 2001, 46, 2002, 59, 397, 2003,...
1998    [432, 2000, 46, 2001, 46, 2002, 46, 2003, 46, ...
1999    [618, 432, 2000, 46, 2001, 46, 2002, 46, 2003,...
Name: Vector, Length: 2000, dtype: object
0       [618, 432, 2000, 46, 2001, 46, 2002, 46, 2003,...
1       [432, 2000, 46, 2001, 46, 2002, 46, 2003, 59, ...
2       [604, 432, 2000, 46, 2001, 46, 2002, 46, 377, ...
3       [432, 2000, 46, 2001, 46, 2002, 46, 2003, 59, ...
4       [432, 2000, 46, 2001, 46, 2002, 59, 397, 2000,...
                              

### Train/predict procedure

In [2]:
from keras import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


# Model parameters
MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 100
EPOCHS = 10
BATCH_SIZE = 64


def split_dataset(df):
    """
    Split the dataset into training and test sets

    :param df: The dataset as a Pandas DataFrame
    :return A tuple of format (X_train, X_test, Y_train, Y_test)
    """

    X = pad_sequences(df['Vector'].values, MAX_SEQUENCE_LENGTH)
    Y = pd.get_dummies(df['Smell']).values
    return train_test_split(X, Y, test_size=0.3, random_state=42)


def train_LSTM(X_train, Y_train, Xs):
    """
    Train a LSTM model for the given training dataset

    :param X_train: The training feature vector
    :param Y_train: The labels vector
    :param Xs: The shape of the training feature vector
    :return The trained model
    """

    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=Xs))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train model
    model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
    return model

In [3]:
# For ConditionalTestLogic
X_train, X_test, Y_train, Y_test = split_dataset(df1)
model = train_LSTM(X_train, Y_train, X_train.shape[1])
accr = model.evaluate(X_test, Y_test)
print('\nTest Set Performance\nLoss: {:0.3f}\nAccuracy: {:0.3f}'.format(accr[0], accr[1]))


Epoch 1/10
22/22 [==============================] - 36s 2s/step - loss: 0.6896 - accuracy: 0.5400
Epoch 2/10
22/22 [==============================] - 23s 1s/step - loss: 0.5953 - accuracy: 0.7050
Epoch 3/10
22/22 [==============================] - 20s 912ms/step - loss: 0.5643 - accuracy: 0.7457
Epoch 4/10
22/22 [==============================] - 20s 887ms/step - loss: 0.4941 - accuracy: 0.7893
Epoch 5/10
22/22 [==============================] - 22s 1s/step - loss: 0.4208 - accuracy: 0.8171
Epoch 6/10
22/22 [==============================] - 21s 973ms/step - loss: 0.3913 - accuracy: 0.8414
Epoch 7/10
22/22 [==============================] - 25s 1s/step - loss: 0.3329 - accuracy: 0.8607
Epoch 8/10
22/22 [==============================] - 23s 1s/step - loss: 0.3453 - accuracy: 0.8650
Epoch 9/10
22/22 [==============================] - 26s 1s/step - loss: 0.3211 - accuracy: 0.8743
Epoch 10/10
19/19 [==============================] - 2s 86ms/step - loss: 0.3653 - accuracy: 0.8633

Test Set

In [4]:
# For ExceptionCatchingThrowing
X_train, X_test, Y_train, Y_test = split_dataset(df2)
model = train_LSTM(X_train, Y_train, X_train.shape[1])
accr = model.evaluate(X_test, Y_test)
print('\nTest Set Performance\nLoss: {:0.3f}\nAccuracy: {:0.3f}'.format(accr[0], accr[1]))

Epoch 1/10
22/22 [==============================] - 22s 992ms/step - loss: 0.6879 - accuracy: 0.5400
Epoch 2/10
22/22 [==============================] - 22s 979ms/step - loss: 0.6251 - accuracy: 0.6714
Epoch 3/10
22/22 [==============================] - 22s 983ms/step - loss: 0.5158 - accuracy: 0.7550
Epoch 4/10
22/22 [==============================] - 22s 980ms/step - loss: 0.4757 - accuracy: 0.7893
Epoch 5/10
22/22 [==============================] - 22s 1s/step - loss: 0.4067 - accuracy: 0.8214
Epoch 6/10
22/22 [==============================] - 21s 976ms/step - loss: 0.3623 - accuracy: 0.8479
Epoch 7/10
22/22 [==============================] - 21s 960ms/step - loss: 0.3082 - accuracy: 0.8779
Epoch 8/10
22/22 [==============================] - 23s 1s/step - loss: 0.3125 - accuracy: 0.8850
Epoch 9/10
22/22 [==============================] - 22s 1s/step - loss: 0.3445 - accuracy: 0.8921
Epoch 10/10
19/19 [==============================] - 2s 104ms/step - loss: 0.3498 - accuracy: 0.8683